In [1]:
!pip install wurlitzer
%load_ext wurlitzer

In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from numba import cuda
import numba
import math
import time

# 1 - comprendre les 2D

In [17]:
@cuda.jit
def kernnelMult(a, b, c):
    # get the position of thread
    i, j = cuda.grid(2)
    print(i, " - ", j)
    if(i < c.shape[0] and  j < c.shape[1]):
         print("worked !")
         # c[i, j] = a[0, j] * b[i, 0] + a[1, j] * b[i, 1] + a[2, j] * b[i, 2]
         tmp = 0
         for k in range(a.shape[0]):
             tmp += a[k, j] * b[i, k]
         c[i, j] = tmp


In [13]:
# Create grid 'a'
a = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

# Create grid 'b'
b = np.array([[9, 8, 7],
              [6, 5, 4],
              [3, 2, 1]])

In [18]:
def multplyGrids(a, b):
    c = np.empty_like(a)

    # 1- CHOIX
    thread_per_block = (2, 2)
    blocks_per_gridX = math.ceil(a.shape[0] / thread_per_block[0])
    blocks_per_gridY = math.ceil(a.shape[0] / thread_per_block[1])
    blocks_per_grid = (blocks_per_gridX, blocks_per_gridY)

    # 2 - Allocation de la mémoire sur le GPU
    a_GPU = cuda.to_device(a)
    b_GPU = cuda.to_device(b)
    c_GPU = cuda.to_device(c)

    # 3 - Exécution du kernel :
    kernnelMult[blocks_per_grid, thread_per_block](a_GPU, b_GPU, c_GPU)

    # 4 - copier le resultat host
    c = c_GPU.copy_to_host()
    return c

print(multplyGrids(a, b))




/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[[9 8 7]
 [6 5 4]
 [3 2 1]]0  -  2
1  -  2
0  -  3
1  -  3
0  -  0
1  -  0
0  -  1
1  -  1
2  -  0
3  -  0
2  -  1
3  -  1
2  -  2
3  -  2
2  -  3
3  -  3
worked !
worked !
worked !
worked !
worked !
worked !
worked !
worked !
worked !

